In [367]:
import requests
from lxml import html as lhtml
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup
from ctypes import c_int
from multiprocessing import Pool,Lock,Value

In [342]:
with open('authors.txt') as authors:
        authors_ids = [int(line.rstrip()) for line in authors]

In [343]:
def get_page(url, attempts = 3):
    for get in range(attempts):
        page = requests.get(url)
        if not page.ok:
            print(page.status_code, page_i)
        else:
            return page
        time.sleep(1)
    print("Error in getting {} page".format(url))

In [375]:
urls=[]

In [376]:
def get_authors_books(author_id):
    #print(author_id)
    book_urls=[]
    path_for_books = '//div[@class="rd-page-listing__two-col_right"]/div[@class="rd-page-listing__products"]/div[contains(@class,rd-listing-product-item-data-wrap)]/div/a/attribute::href'
    path_for_num = '//div[@class="rd-page-listing__three-col"]/div/div/span[@class="rd-listing-count__total"]/text()'
    page = get_page('https://www.respublica.ru/authors/{}?page=1'.format(author_id))
    tree = lhtml.fromstring(page.text)
    urls = tree.xpath(path_for_books)
    num_of_items = tree.xpath(path_for_num)
    for url in urls:
        book_urls.append('https://www.respublica.ru' + url)
    num_of_items = int(num_of_items[0].rstrip())
    for page_num in range(2,ceil(num_of_items/22)+1):
        page = get_page('https://www.respublica.ru/authors/{}?page={}'.format(author_id,page_num))
        tree = lhtml.fromstring(page.text)
        urls = tree.xpath(path_for_books)
        for url in urls:
            book_urls.append('https://www.respublica.ru' + url)
    return book_urls

In [378]:
%%time
lock = Lock()
ticker = Value(c_int,0)

def logger(author_id):
    get_authors_books(author_id)
    with lock:
        global ticker
        ticker.value+=1
        print("get author number ",ticker.value)
        
with Pool(10) as pool:
    pages = pool.map(logger, authors_ids)
pool.join()

get author number  1
get author number  2
get author number  3
get author number  4
get author number  5
get author number  6
get author number  7
get author number  8
get author number  9
get author number  10
get author number  11
get author number  12
get author number  13
get author number  14
get author number  15
get author number  16
get author number  17
get author number  18
get author number  19
get author number  20
get author number  21
get author number  22
get author number  23
get author number  24
get author number  25
get author number  26
get author number  27
get author number  28
get author number  29
get author number  30
get author number  31
get author number  32
get author number  33
get author number  34
get author number  35
CPU times: user 114 ms, sys: 98.1 ms, total: 212 ms
Wall time: 22.6 s


In [371]:
book_urls = set(book_urls)

In [372]:
len(book_urls)

0

In [373]:
def get_card(url):
    card = dict()
    page = get_page(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    # Разделим страницу на три части
    left_attrs = soup.find('div',class_='rd-page-product__col-left')
    right_attrs = soup.find('div',class_='rd-page-product__col-right')
    bottom = soup.find('div',class_="rd-page-product__row rd-page-product__desc")
    card['ID'] = left_attrs.find('span',itemprop = "sku").text
    card['Цена'] = int(right_attrs.find('span',class_='num').text)
    card['Название'] = left_attrs.find('h1',itemprop = "name").text
    card['URL'] = url
    preview = left_attrs.find('a',class_ = "download-pdf")
    if preview is not None:
        card['Превью'] = 'https://www.respublica.ru' + preview.attrs['href']
    card['Изображение'] = 'https://www.respublica.ru' + \
        left_attrs.find('img',class_="rd-page-product__img").attrs['src']
    card['Автор'] = left_attrs.find('a',itemprop = "brand").text
    a = soup.find_all('span',class_="rd-page-breadcrumbs-item")
    a = [e.text for e in soup.find_all('span',class_="rd-page-breadcrumbs-item")]
    card['Категория'] = '; '.join([item[1:] for item in a])
    rating_info = right_attrs.find('span', itemprop = "aggregateRating")
    if rating_info is not None:
        card['Количество отзывов'] = int(rating_info.find('meta',itemprop = "reviewCount").attrs['content'])
        card['Количество оценок'] = int(rating_info.find('meta',itemprop = "ratingCount").attrs['content'])
        card['Оценка'] = float(rating_info.find('meta',itemprop = "ratingValue").attrs['content'])
    else:
        card['Количество отзывов'] = 0
        card['Количество оценок'] = 0
        card['Оценка'] = 0
    old_price = right_attrs.find('div',class_="rd-page-product__price-old")
    if old_price is not None:
        card['Цена(старая)'] = int(old_price.find_next('span',class_="prev").text[:-5])
    button_text = right_attrs.find('span',class_="rd-page-product__buy-text").text
    if button_text == "Купить" or button_text == "Купить со скидкой":
        card['Наличие'] = True
    else:
        card['Наличие'] = False
    description = bottom.find('div',class_="rd-page-product__col-left")
    text_tag = description.find('p')
    if text_tag is not None:
        card['Описание'] = text_tag.text
    else:
        card['Описание'] = text_tag = description.find('div', itemprop = "description").text    
    table = bottom.find('div',class_="rd-page-product__desc-params")
    rows = table.find_all('p',class_="rd-page-product__desc-param")
    for row in rows:
        name = row.find('span',itemprop="name").text
        non_url_value = row.find('span',itemprop="value")
        if non_url_value is None:
            card[name] = row.find('a',itemprop="value").text
        else:
            card[name] = non_url_value.text
    return card

In [340]:
get_card('https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/532807-myorfi-d-pyshechka-m-o')

{'ID': '532807',
 'Цена': 520,
 'Название': 'Пышечка',
 'URL': 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/532807-myorfi-d-pyshechka-m-o',
 'Изображение': 'https://www.respublica.ru/uploads/01/00/00/dq/bv/large_996ef85554b87d57.jpg',
 'Автор': 'Джули Мёрфи',
 'Категория': 'Книги; Художественная литература; Современная проза',
 'Количество отзывов': 0,
 'Количество оценок': 0,
 'Оценка': 0,
 'Наличие': True,
 'Описание': '"Эта книга вдохновляет. Вместо того чтобы дарить детям комплексы, подарите им эту книгу. Пусть она, а не модели Victoria\'s Secret будут для них примером". - THE GUARDIAN',
 'ISBN': '978-5-6042196-4-5',
 'Издательство': 'Popcorn Books',
 'Обложка': 'Мягкая',
 'Формат': '14 х 21',
 'Количество страниц': '456',
 'Год издания': '2019',
 'Язык': 'Русский',
 'Вес, г': '464',
 'Раздел': 'Зарубежные',
 'Вид бумаги': 'Офсетная'}